## Import all Required library

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
#import sys
#sys.path.append('/content/drive/My Drive/Colab Notebooks/Assignment3_nlp')
import utils_final
#!python "/content/drive/My Drive/Colab Notebooks/utils_final.py"



2024-03-08 11:18:36.478943: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
 !pip install datasets
 !pip install seqeval
 !pip install accelerate==0.21.0
 !pip install transformers[torch]
 !pip install accelerate -U
!pip install torch - upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=bad2e672079ab96857b2f525e0670a48a42f6cf392b96effd090f4a2cf151736
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0
ERROR: In

In [1]:
import time
import pandas as pd
import transformers
import numpy as np
from transformers import AutoTokenizer,AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_metric
from datasets import Dataset
from utils_final import read_data_as_sentence,map_labels_in_dataframe,tokenize_and_align_labels,get_label_mapping,get_labels_from_map,load_srl_model,load_dataset,compute_metrics,write_predictions_to_csv,compute_evaluation_metrics_from_csv
from sklearn.metrics import precision_recall_fscore_support
from main_bert_srl import main, define_args

2024-03-08 11:47:40.917409: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Preprocessing data

Read data files and save them as dataframe.\
\
Use `read_data_as_sentence` function from `utils.py` to read files as Dataframe.\

`read_data_as_sentence` need two input:
1. path of conllu file.
2. path for save datarame

In [6]:
train_data = read_data_as_sentence('data/en_ewt-up-train.conllu', 'data/en_ewt-up-train.preprocessed.csv', mode='basic')
dev_data = read_data_as_sentence('data/en_ewt-up-dev.conllu', 'data/en_ewt-up-dev.preprocessed.csv',mode='basic')
test_data = read_data_as_sentence('data/en_ewt-up-test.conllu', 'data/en_ewt-up-test.preprocessed.csv',mode='basic')

In [ ]:
print(len(train_data))

40482


Show 20 input form and their gold list in test data

In [ ]:
for i in range(20):
    print(f"{test_data.input_form[i]}\t:\t{test_data.argument[i]}")

NameError: name 'test_data' is not defined

Head of test data after process

In [ ]:
test_data.head()

,input_form,argument
0,"[What, if, Google, Morphed, Into, GoogleOS, ?,...","[_, _, ARG1, _, _, ARG2, _, None, None, None, ..."
1,"[What, if, Google, expanded, on, its, search, ...","[_, _, ARG0, _, _, _, _, _, _, _, _, _, _, _, ..."
2,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."
3,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, ARGM-DIS, _, _, ARG1, _, _, _,..."
4,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4799 entries, 0 to 4798
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   input_form  4799 non-null   object
 1   argument    4799 non-null   object
dtypes: object(2)
memory usage: 75.1+ KB


## Importing the model and tokenizer

In [7]:
model_checkpoint = "distilbert-base-uncased"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Checking the sentence representation

In [9]:
example = test_data['input_form'][1]
print(example)

['What', 'if', 'Google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e-mail', ')', 'wares', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'expanded', 'Google', 'expanded', 'on']


The sentence contains the [SEP] special token followed by the predicate. Therefore, the parameter `add_special_tokens` is set to True so that the index is converted to 102 accordingly and is not treated as another word. \
In addition, the sentence is already split into tokens, to the parameter `is_split_into_words` is also set to True

In [10]:
tokenizer(example,add_special_tokens=True,is_split_into_words=True)

{'input_ids': [101, 2054, 2065, 8224, 4423, 2006, 2049, 3945, 1011, 3194, 1006, 1998, 2085, 1041, 1011, 5653, 1007, 16283, 2015, 2046, 1037, 2440, 1011, 26712, 4082, 2291, 1029, 102, 4423, 8224, 4423, 2006, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenized_input = tokenizer(example,add_special_tokens=True,is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'what', 'if', 'google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e', '-', 'mail', ')', 'ware', '##s', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'google', 'expanded', 'on', '[SEP]']


## Tokenizing and preparing input for the model

Getting the mapping of all possible arguments across all datasets to a numerical value with the `get_label_mapping` function.\
None value stays as None to be mapped to the special token in the model.

In [ ]:
label_map = get_label_mapping(train_data, test_data, dev_data)

In [ ]:
print(label_map)

{'_': 0, 'ARG0': 1, 'ARG1': 2, 'ARG1-DSP': 3, 'ARG2': 4, 'ARG3': 5, 'ARG4': 6, 'ARG5': 7, 'ARGA': 8, 'ARGM-ADJ': 9, 'ARGM-ADV': 10, 'ARGM-CAU': 11, 'ARGM-COM': 12, 'ARGM-CXN': 13, 'ARGM-DIR': 14, 'ARGM-DIS': 15, 'ARGM-EXT': 16, 'ARGM-GOL': 17, 'ARGM-LOC': 18, 'ARGM-LVB': 19, 'ARGM-MNR': 20, 'ARGM-MOD': 21, 'ARGM-NEG': 22, 'ARGM-PRD': 23, 'ARGM-PRP': 24, 'ARGM-PRR': 25, 'ARGM-REC': 26, 'ARGM-TMP': 27, 'C-ARG0': 28, 'C-ARG1': 29, 'C-ARG1-DSP': 30, 'C-ARG2': 31, 'C-ARG3': 32, 'C-ARG4': 33, 'C-ARGM-ADV': 34, 'C-ARGM-COM': 35, 'C-ARGM-CXN': 36, 'C-ARGM-DIR': 37, 'C-ARGM-EXT': 38, 'C-ARGM-GOL': 39, 'C-ARGM-LOC': 40, 'C-ARGM-MNR': 41, 'C-ARGM-PRP': 42, 'C-ARGM-PRR': 43, 'C-ARGM-TMP': 44, 'R-ARG0': 45, 'R-ARG1': 46, 'R-ARG2': 47, 'R-ARG3': 48, 'R-ARG4': 49, 'R-ARGM-ADJ': 50, 'R-ARGM-ADV': 51, 'R-ARGM-CAU': 52, 'R-ARGM-COM': 53, 'R-ARGM-DIR': 54, 'R-ARGM-GOL': 55, 'R-ARGM-LOC': 56, 'R-ARGM-MNR': 57, 'R-ARGM-TMP': 58, None: None}


Converting the labels in the df to numerical values for the language model with `map_labels_in_dataframe` function. The label_map dictionary from the function above is needed to map the arguments to their value.\
Add a new column to the df matching the arguments to label numbers. 0 stands for '_' (no argument) and the rest of the arguments are alphabetically ordered. \
*None* label will be mapped to the *[SEP]* token.


In [ ]:
train_data = map_labels_in_dataframe(train_data,label_map)
dev_data = map_labels_in_dataframe(dev_data,label_map)
test_data = map_labels_in_dataframe(test_data,label_map)

Checking the head to confirm the labels were correctly converted:

In [ ]:
test_data.head()

,input_form,argument,mapped_labels
0,"[What, if, Google, Morphed, Into, GoogleOS, ?,...","[_, _, ARG1, _, _, ARG2, _, None, None, None, ...","[0, 0, 2, 0, 0, 4, 0, None, None, None, None]"
1,"[What, if, Google, expanded, on, its, search, ...","[_, _, ARG0, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, ..."
2,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, ARGM-DIS, _, _, ARG1, _, _, _,...","[0, 0, 0, 0, 0, 15, 0, 0, 2, 0, 0, 0, 0, 4, 0,..."
4,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Use `tokenize_and_align_labels` function to tokenize train, test, and dev dataframe. Padding is applied to make sure all input is the same length for the model.

In [ ]:
tokenized_test = tokenize_and_align_labels(tokenizer, test_data, label_all_tokens=True)
tokenized_train = tokenize_and_align_labels(tokenizer, train_data, label_all_tokens=True)
tokenized_dev = tokenize_and_align_labels(tokenizer, dev_data, label_all_tokens=True)

IndexError: list index out of range

The input for the model has the corresponding special token [CLS] followed by the tokenized sentence, the special token [SEP], the predicate and the final [SEP] token. \
The numerical labels to be fed to the model correspond to the tokenized sentence.\
The input is padded so that every vector is of the same length, including the labels and the attention mask.

In [ ]:
print(tokenizer.convert_ids_to_tokens(tokenized_test["input_ids"][0]))
print(tokenized_test["attention_mask"][0])
print(tokenized_test["input_ids"][0])
print(tokenized_test["labels"][0])

['[CLS]', 'what', 'if', 'google', 'mor', '##ph', '##ed', 'into', 'google', '##os', '?', '[SEP]', 'google', 'mor', '##ph', '##ed', 'into', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 

Confirming all the tokens contain a label and the attention mask also matches the length of the input

In [ ]:
print(len(tokenized_test["input_ids"][0]),len(tokenized_test["labels"][0]),len(tokenized_test["attention_mask"][0]))

100 100 100


Converting the tokenized data to datasets format with the function `load_dataset`

In [ ]:
dataset_train = load_dataset(tokenized_train)
dataset_dev = load_dataset(tokenized_dev)
dataset_test = load_dataset(tokenized_test)

## Fine-tuning the model

Reducing the size of the dataset for a mini test

In [ ]:
small_train_dataset = dataset_train.shuffle(seed=42)#.select(range(1000))
small_eval_dataset = dataset_dev.shuffle(seed=42)#.select(range(1000))
small_test_dataset = dataset_test.shuffle(seed=42)#.select(range(1000))

Getting the labels that will be predicted by the model with the `get_labels_from_map`function

In [ ]:
label_list = get_labels_from_map(label_map)

Loading the model for semantic role labelling task with function `load_srl_model` to get the model, its name and the arguments necessary for training. \
The model selected is **distilbert-base-uncased**

In [ ]:
model, model_name, args = load_srl_model(model_checkpoint, label_list)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model_name)

distilbert-base-uncased


Load the seqeval metric to compute the metrics from the predictions

In [ ]:
# metric = load_metric(sklearn.metric_precision_recall_fscore_support)

metric.compute(predictions=[label_list], references=[label_list])

Passing the arguments along with the datasets to the `trainer` function to fine-tune the model for semantic role labelling with `trainer.train()`

In [ ]:
trainer = Trainer(
        model,
        args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=lambda p: compute_metrics(*p, label_list)
    )
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.131300,0.135953,0.332628,0.312226,0.315973,0.959560
2,0.099400,0.117762,0.392546,0.357416,0.364806,0.964388
3,0.082300,0.115561,0.419814,0.372430,0.379407,0.964827


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

TrainOutput(global_step=7593, training_loss=0.1311375363517087, metrics={'train_runtime': 2139.9686, 'train_samples_per_second': 56.751, 'train_steps_per_second': 3.548, 'total_flos': 5677163855723412.0, 'train_loss': 0.1311375363517087, 'epoch': 3.0})

Evaluate a model fine-tuned for semantic role labelling with `trainer.evaluate()`

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.11556050926446915,
 'eval_precision': 0.4198138658245874,
 'eval_recall': 0.37242957940686916,
 'eval_f1': 0.3794074901707871,
 'eval_accuracy': 0.9648266721746015,
 'eval_runtime': 25.2217,
 'eval_samples_per_second': 197.33,
 'eval_steps_per_second': 12.37,
 'epoch': 3.0}

After training is finished, the precision/recall/f1 for each category can be computed. \
The same function `compute_metrics` is applied on the result of the predict method.

In [ ]:
predictions, labels, _ = trainer.predict(small_eval_dataset)
results = compute_metrics(predictions, labels, label_list)
results

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

{'precision': 0.4198138658245874,
 'recall': 0.37242957940686916,
 'f1': 0.3794074901707871,
 'accuracy': 0.9648266721746015}

Writing the predictions together with the gold labels to a csv file with the function `write_predictions_to_csv` so that the metrics per class can be computed with the `compute_evaluation_metrics_from_csv` function.

In [ ]:
results_file = "predictions.csv"
write_predictions_to_csv(predictions, labels, label_list, results_file)
classification_report = compute_evaluation_metrics_from_csv("predictions.csv")
print(classification_report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        ARG0       0.86      0.83      0.84      1931
        ARG1       0.84      0.80      0.82      3851
        ARG2       0.68      0.70      0.69      1397
        ARG3       0.00      0.00      0.00         0
        ARG4       0.43      0.71      0.54        41
        ARG5       0.00      0.00      0.00         0
    ARGM-ADJ       0.73      0.74      0.73       273
    ARGM-ADV       0.55      0.66      0.60       431
    ARGM-CAU       0.57      0.61      0.59        71
    ARGM-COM       0.07      0.17      0.10         6
    ARGM-CXN       0.14      0.67      0.24         3
    ARGM-DIR       0.47      0.40      0.43        65
    ARGM-DIS       0.69      0.76      0.72       186
    ARGM-EXT       0.76      0.78      0.77       120
    ARGM-GOL       0.00      0.00      0.00         0
    ARGM-LOC       0.57      0.58      0.57       359
    ARGM-LVB       0.83      0.83      0.83        75
    ARGM-MNR       0.56    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
save_directory = '/content/drive/My Drive/Colab Notebooks/Assignment3_nlp/'
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('/content/drive/My Drive/Colab Notebooks/Assignment3_nlp/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/Assignment3_nlp/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/Assignment3_nlp/vocab.txt',
 '/content/drive/My Drive/Colab Notebooks/Assignment3_nlp/added_tokens.json',
 '/content/drive/My Drive/Colab Notebooks/Assignment3_nlp/tokenizer.json')

In [2]:
args=define_args(mode='advanced')

In [3]:
main(args,mode='advanced', data_range=100)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,3.535963,0.021727,0.022805,0.020511,0.651173


***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
/Users/g/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/g/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/g/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, ms

/Users/g/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/g/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/g/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/g/anaconda3/lib/python3.11/site-packages/

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]